In [1]:
import pandas as pd
import numpy as np
from src.utils import setup_folders, return_static_features, return_dynamic_features
from src.visualization.evaluate import calculate_metric_horizons, calculate_metric_horizon_windows, calculate_metric_horizons_all_models
from src.visualization.visualize import plot_horizons, plot_scatterplot, plot_test_day, plot_mean_std_error_multiple_models, plot_ridges_multiple_models
from src.features.build_features import build_features, build_features_LSTM
import json

In [2]:
def load_param_json(plant, model, tech):
    # agr
    filename = f'../src/models/deterministic/params/params_{model}_{plant}.json'
    f = open(filename)
    params = json.load(f)
    f.close()
    
    return(params[tech])

In [3]:
params = load_param_json("HPP1", "LSTM", 'wind')

In [4]:
list(params.keys())

['activation',
 'n_lag',
 'epochs',
 'batch_size',
 'lr',
 'neurons_l1',
 'neurons_l2',
 'neurons_l3']

In [5]:
# We add these to a dataframe with a multikey index
outer_index = ['LGB', 'LSTM', 'RF']
#inner_index = list(params.keys())
tuples = []
for outer in outer_index:
    params = load_param_json("HPP1", outer, 'wind')
    inner_index = list(params.keys())
    for inner in inner_index:
        tuples.append((outer,inner))
        
index = pd.MultiIndex.from_tuples(tuples, names=["Model", "Parameter"])

In [6]:
index

MultiIndex([( 'LGB',         'max_depth'),
            ( 'LGB',  'feature_fraction'),
            ( 'LGB',        'num_leaves'),
            ( 'LGB',    'num_iterations'),
            ('LSTM',        'activation'),
            ('LSTM',             'n_lag'),
            ('LSTM',            'epochs'),
            ('LSTM',        'batch_size'),
            ('LSTM',                'lr'),
            ('LSTM',        'neurons_l1'),
            ('LSTM',        'neurons_l2'),
            ('LSTM',        'neurons_l3'),
            (  'RF',         'max_depth'),
            (  'RF',  'min_samples_leaf'),
            (  'RF', 'min_samples_split'),
            (  'RF',      'n_estimators')],
           names=['Model', 'Parameter'])

In [7]:
list(params.values())

[31, 1, 2, 194]

In [8]:
plants = ["Nazeerabad", "HPP1", "HPP2", "HPP3"]
models = ['LGB', 'LSTM', 'RF']
def build_table(tech):
    dict_results = {}
    for plant in plants:
        param_values = []
        for model in models:
            params = load_param_json(plant, model, tech)
            param_values.append(list(params.values()))
            
        dict_results[plant] = np.concatenate(param_values)
        
    table = pd.DataFrame(dict_results, index = index)
    return(table)

In [9]:
table = build_table('wind')
table

Nazeerabad   HPP1   HPP2   HPP3
Model Parameter                                        
LGB   max_depth                  8   16.0    5.0    6.0
      feature_fraction           1    0.4    0.2    0.8
      num_leaves                61    4.0    2.0    3.0
      num_iterations            62  423.0  288.0  404.0
LSTM  activation              tanh   tanh   tanh   tanh
      n_lag                    144    144    144    144
      epochs                    20     20     20     20
      batch_size              1500   1500   1500   1500
      lr                      0.01   0.01   0.01   0.01
      neurons_l1                75    125     25    125
      neurons_l2                50     75     50    100
      neurons_l3                50     70     30     30
RF    max_depth                 90     31     41     59
      min_samples_leaf           1      1      1      1
      min_samples_split          2      2      2      2
      n_estimators             197    194     27    116

In [10]:
path_out_tables = f"../reports/results/All Plants/Tables"
setup_folders(path_out_tables)

In [11]:
for tech in ['wind', 'solar', 'agr']:
    table = build_table(tech)
    table.to_csv(f"{path_out_tables}/table_params_{tech}.csv", sep = ";")